In [1]:
import flashtext

In [17]:
import pandas as pd
import re

In [3]:
gaz = pd.read_excel('rnd3/gaz_fe_new.xlsx')

In [127]:
reps = pd.read_excel('replacements.xlsx')

In [128]:
reps.fillna('', inplace=True)
reps.columns = ['fstr', 'rstr']

In [129]:
pgr = re.compile(r'Page\w*')
spr = re.compile(r'[ ]+')
def cleanup(text):
    text = text.replace('~','').replace('#','').replace('<br />','').replace('%', '').replace('Milestone300', '')
    text = pgr.sub('', text)
    text = spr.sub(' ', text)
    text = text.replace('\n ','\n')
    return text

In [130]:
clean_text = [cleanup(t) for t in gaz.text]

In [41]:
gaz.text = clean_text

In [218]:
toproc = flashtext.KeywordProcessor()
kp = flashtext.KeywordProcessor()
kp.non_word_boundaries.update(achars)

In [219]:
for row in reps.itertuples():
    rp = row.rstr
    if not rp:
        rp = ' '
    if rp.endswith(' ' + 'ل'):
        toproc.add_keyword(row.fstr, rp)
    else:
        kp.add_keyword(row.fstr, rp)

In [220]:
exc = pd.read_excel('exceptions.xlsx')

In [230]:
ispat = re.compile(r'\W(هو|هي|وهو|وهي|من|اسم)\W')

In [213]:
def replacer(text):
    t = toproc.replace_keywords(text)
    t = kp.replace_keywords(t)
    return t

In [231]:
newtypes = []
for row in gaz.itertuples():
    tx = row.text
    _, _, findpart = tx.partition(':')
    internal = replacer(findpart)
    ts = ''
    if internal:
        ts = internal.split()[0]
    if ts in exc.pattern.values:
        ts = ''
        parts = ispat.split(findpart)
        if len(parts) > 1:
            internal = replacer(parts[1])
            if internal.split():
                ts = internal.split()[0]
    newtypes.append(ts)

In [232]:
newtypes

['',
 'جبل',
 'إن',
 'كسرى،',
 'أثرب',
 '',
 'جبل',
 '',
 'لنهر',
 'بلد',
 '',
 'موضع',
 'مدينة',
 'خوز',
 '',
 '',
 'إقليم',
 '',
 '',
 '',
 '',
 'قلعة',
 'أرض',
 'ماءة',
 '',
 'أضلاع',
 'بلد',
 '',
 'جبل',
 'الموضع',
 '',
 'نهران',
 '',
 'موضع',
 '',
 'قرية',
 'الجبل',
 'كانت',
 'قرية',
 'مدينة',
 'إن',
 'موضع',
 'منها',
 'قرية',
 'ليس',
 '',
 'واديان',
 'محلة',
 'قرية',
 'إن',
 'واد',
 'مدينة',
 'موضع',
 'مدينة',
 '',
 'حوش',
 '',
 '',
 'بركة',
 'كورة',
 'جبل',
 'الصقع',
 'بلد',
 '',
 '',
 'الطريق',
 'بين',
 'قرية',
 '',
 'الرافقة',
 'فيها',
 'موضع',
 'الرقاق',
 'تعجيم',
 'رمادة',
 'مدينة',
 'أنشدني',
 'زاب',
 'جزيرة',
 'مجاز',
 'موضع',
 'النير،',
 'ربض',
 'موضع',
 'قرية',
 'أعلى',
 'أرض',
 'مدينة',
 '',
 'قرية',
 'بلدة',
 'موضع',
 'قرية',
 '',
 'قرية',
 '',
 'محلة',
 '',
 'قرية',
 '',
 'كورة',
 'كورة',
 'وذكر',
 '',
 '',
 'بلدة',
 'نهر',
 'طبرستان',
 'بليدة',
 'واد',
 '',
 'والعرب',
 'لا',
 'الغراء',
 'المنخفض',
 'سميت',
 'يجمع',
 'قرية',
 'فلاليج',
 'موضع',
 'القادس',
 'ثغر',
 'مو

In [233]:
def remove_leading_al(s):
    if s.startswith('ال'):
        return s[2:]
    return s

In [234]:
newtypes = [remove_leading_al(w) for w in newtypes]

In [235]:
gaz.Type = newtypes

In [186]:
filters = pd.read_excel('rnd3/Filtering.xlsx')

In [187]:
fwds = list(filters.Delete)

In [236]:
gazf = gaz[~(gaz.name.isin(fwds))]

In [241]:
gazf.to_excel('processed-feb23-filtered.xlsx')